In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import crack_dataset as DS
import copy
import os
import time
import logging
print("PyTorch Version: ",torch.__version__)
import sys
print(sys.argv[0])

PyTorch Version:  1.10.0a0+0aef44c
/opt/conda/lib/python3.8/site-packages/ipykernel_launcher.py


In [ ]:
def getDataSet(patch_size, batch_size, workers=8):
    """
    This function is the one from model training framework
    """
    class Args:
      dataset_path = "/storage/data/classification_dataset_balanced/"
      patch_size = 1
      batch_size = 1
      workers = 1
      def __init__(self, patch_size, batch_size, workers):
        self.patch_size = patch_size
        self.batch_size = batch_size
        self.workers = workers
    args = Args(patch_size, batch_size, workers)
    dataset = DS.CODEBRIM(torch.cuda.is_available(),args)
    dataLoaders = {'train': dataset.train_loader, 'val': dataset.val_loader, 'test':dataset.test_loader}
    return dataLoaders

In [ ]:
"""
This cell aims to get the inital models for loading the state of trained models
"""
from ZenNAS.ZenNet import get_ZenNet
from ZenNAS.PlainNet import basic_blocks
def get_ZenNet_pretrained(model_name, num_classes = 6):
    model = get_ZenNet(model_name, pretrained=True)
    
    # adjust the last layer to adapt to the new class number
    model.fc_linear = basic_blocks.Linear(in_channels=model.fc_linear.in_channels, out_channels=num_classes)
    return model

class Resnet(nn.Module):
  """
  Resnet model class modified for multi-target Crack dataset
  """
  def __init__(self):
    super(Resnet, self).__init__()
    self.model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', pretrained=True)
    self.model.fc = nn.Linear(2048,6) # modify the output layer
  def forward(self, x):
    """
    forward step
    """
    x = self.model(x)
    x = torch.sigmoid(x)  # for binary output
    return x
  def _initialize_weights(self):
    """
    initialzie the parameters
    """
    print("initialize parameters")
    for m in self.modules():
      if isinstance(m, nn.Conv2d):
        nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
        if m.bias is not None:
            nn.init.constant_(m.bias, 0)
      elif isinstance(m, nn.Linear):
        nn.init.normal_(m.weight, 0, 0.01)  # initialize with normal distribution
        nn.init.constant_(m.bias, 0)


def get_resnet50(num_class=6, pretrained=True):
    import torchvision.models as models
    # load vgg net from torchhub
    model = models.resnet50(pretrained=pretrained)  # Resnet-50
    model.fc = nn.Linear(in_features=model.fc.in_features, out_features=num_class, bias=True) # modify the output layer

    return model

In [ ]:
import argparse
import cv2
import numpy as np
import torch
from PIL import Image
from torchvision import models
from pytorch_grad_cam import GradCAM, \
    ScoreCAM, \
    GradCAMPlusPlus, \
    AblationCAM, \
    XGradCAM, \
    EigenCAM, \
    EigenGradCAM, \
    LayerCAM, \
    FullGrad
from pytorch_grad_cam import GuidedBackpropReLUModel
from pytorch_grad_cam.utils.image import show_cam_on_image, \
    deprocess_image, \
    preprocess_image
import torchvision.transforms as transforms


def get_args():
    """
    Set and get the arguments for CAM
    """
    parser = argparse.ArgumentParser()  
    parser.add_argument('--use-cuda', action='store_true', default=False,
                        help='Use NVIDIA GPU acceleration')
    parser.add_argument(
        '--image-path',
        type=str,
        default='./examples/both.png',
        help='Input image path')
    parser.add_argument('--aug_smooth', action='store_true',
                        help='Apply test time augmentation to smooth the CAM')
    parser.add_argument(
        '--eigen_smooth',
        action='store_true',
        help='Reduce noise by taking the first principle componenet'
        'of cam_weights*activations')
    parser.add_argument('--method', type=str, default='gradcam',
                        choices=['gradcam', 'gradcam++',
                                 'scorecam', 'xgradcam',
                                 'ablationcam', 'eigencam',
                                 'eigengradcam', 'layercam', 'fullgrad'],
                        help='Can be gradcam/gradcam++/scorecam/xgradcam'
                             '/ablationcam/eigencam/eigengradcam/layercam')

    #args = parser.parse_args()
    args, unknown = parser.parse_known_args()
    args.use_cuda = args.use_cuda and torch.cuda.is_available()
    if args.use_cuda:
        print('Using GPU for acceleration')
    else:
        print('Using CPU for computation')

    return args

dataLoaders = getDataSet(224, 1)
#tags for distinguishing and storing images
datatags = ['val', 'test']
for datatag in datatags:
    for i, sample in enumerate(dataLoaders[datatag]):
        args = get_args()
        methods = \
            {"gradcam": GradCAM,
             "scorecam": ScoreCAM,
             "gradcam++": GradCAMPlusPlus,
             "ablationcam": AblationCAM,
             "xgradcam": XGradCAM,
             "eigencam": EigenCAM,
             "eigengradcam": EigenGradCAM,
             "layercam": LayerCAM,
             "fullgrad": FullGrad}
        inputs, labels = sample
                
        #get image for CAM
        toPILTransform = transforms.ToPILImage()
        rgb_img = toPILTransform(inputs[0])
        rgb_img.save(f'{datatag}_{i}_original_img.jpg')
        rgb_img = np.float32(rgb_img) / 255
        
        #get tensor for CAM
        normalize = transforms.Normalize(mean=[ 0.499, 0.559, 0.535], std=[0.021, 0.018, 0.019])
        input_tensor = normalize(inputs)
        
        #collet models for testing their Class Activation Map
        model_list = []
        
        #get resnet50 that is not pretrained
        model = get_resnet50(num_class=6, pretrained=True)
        state_dict = torch.load('./resnet-50_not_pretrained/hard.pth')
        from collections import OrderedDict
        new_state_dict = OrderedDict()
        for k, v in state_dict.items():
            name = k[6:] # remove `module.`
            new_state_dict[name] = v
        model.load_state_dict(new_state_dict)
        model_list.append(model)
        
        #get resnet50 that is pretrained
        model = get_resnet50(num_class=6, pretrained=True)
        state_dict = torch.load('./resnet-50_pretrained/hard.pth')
        model.load_state_dict(state_dict)
        model_list.append(model)
        
        #get ZenNas that is not pretrained
        paras_path = "./hard_ZenNas_withoutPretrain.pth"
        class_num = 6
        model = get_ZenNet_pretrained('zennet_imagenet1k_flops400M_SE_res224', num_classes=class_num)
        model_list.append(model)
 
        #get ZenNas that is pretrained
        paras_path = "./hard_ZenNas_withPretrain.pth"
        class_num = 6
        model = get_ZenNet_pretrained('zennet_imagenet1k_flops400M_SE_res224', num_classes=class_num)
        model_list.append(model)       
        
        #tags for distinguishing and storing images
        tag_list = ['resnet-50_not_pretrained','resnet-50_pretrained','ZenNas_withoutPretrain','ZenNas_withPretrain']
        j = 0
        for model in model_list:
            tag = tag_list[j]
            #The last covolution layer of these two kinds of models are different
            if tag == 'resnet-50_not_pretrained' or tag == 'resnet-50_pretrained':
                target_layers = [model.layer4[-1]]
            else:
                target_layers = [model._modules['module_list'][5]]

            # If None, returns the map for the highest scoring category.
            # Otherwise, targets the requested category.
            target_category = None
        
            # Using the with statement ensures the context is freed, and you can
            # recreate different CAM objects in a loop.
            cam_algorithm = methods[args.method]
            #cam_algorithm = methods["ablationcam"]
            with cam_algorithm(model=model,
                               target_layers=target_layers,
                               use_cuda=args.use_cuda) as cam:
        
                # AblationCAM and ScoreCAM have batched implementations.
                # You can override the internal batch size for faster computation.
                cam.batch_size = 32
        
                grayscale_cam = cam(input_tensor=input_tensor,
                                    target_category=target_category,
                                    aug_smooth=args.aug_smooth,
                                    eigen_smooth=args.eigen_smooth)
        
                # Here grayscale_cam has only one image in the batch
                grayscale_cam = grayscale_cam[0, :]
        
                cam_image = show_cam_on_image(rgb_img, grayscale_cam, use_rgb=True)
        
                # cam_image is RGB encoded whereas "cv2.imwrite" requires BGR encoding.
                cam_image = cv2.cvtColor(cam_image, cv2.COLOR_RGB2BGR)
        
            gb_model = GuidedBackpropReLUModel(model=model, use_cuda=args.use_cuda)
            gb = gb_model(input_tensor, target_category=target_category)
        
            cam_mask = cv2.merge([grayscale_cam, grayscale_cam, grayscale_cam])
            cam_gb = deprocess_image(cam_mask * gb)
            gb = deprocess_image(gb)
            cv2.imwrite(f'{datatag}_{i}_{j}_{tag}_{args.method}_cam.jpg', cam_image)
            cv2.imwrite(f'{datatag}_{i}_{j}_{tag}_{args.method}_gb.jpg', gb)
            cv2.imwrite(f'{datatag}_{i}_{j}_{tag}_{args.method}_cam_gb.jpg', cam_gb)
            j = j + 1
    

Using CPU for computation


Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


ImportError: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html

---debug use_se in SuperResIDWE1K7(16,40,2,40,1)
---debug use_se in SuperResIDWE1K7(40,64,2,64,1)
---debug use_se in SuperResIDWE4K7(64,96,2,96,5)
---debug use_se in SuperResIDWE2K7(96,224,2,224,5)
loading pretrained parameters...
---debug use_se in SuperResIDWE1K7(16,40,2,40,1)
---debug use_se in SuperResIDWE1K7(40,64,2,64,1)
---debug use_se in SuperResIDWE4K7(64,96,2,96,5)
---debug use_se in SuperResIDWE2K7(96,224,2,224,5)
loading pretrained parameters...
Using CPU for computation
---debug use_se in SuperResIDWE1K7(16,40,2,40,1)
---debug use_se in SuperResIDWE1K7(40,64,2,64,1)
---debug use_se in SuperResIDWE4K7(64,96,2,96,5)
---debug use_se in SuperResIDWE2K7(96,224,2,224,5)
loading pretrained parameters...
---debug use_se in SuperResIDWE1K7(16,40,2,40,1)
---debug use_se in SuperResIDWE1K7(40,64,2,64,1)
---debug use_se in SuperResIDWE4K7(64,96,2,96,5)
---debug use_se in SuperResIDWE2K7(96,224,2,224,5)
loading pretrained parameters...
Using CPU for computation
---debug use_se in Super

In [1]:
import zipfile
import os
def compress_file(zipfilename, dirname):      
    if os.path.isfile(dirname):
        with zipfile.ZipFile(zipfilename, 'w') as z:
            z.write(dirname)
    else:
        with zipfile.ZipFile(zipfilename, 'w') as z:
            for root, dirs, files in os.walk(dirname):
                for single_file in files:
                    if single_file != zipfilename:
                        filepath = os.path.join(root, single_file)
                        z.write(filepath)

compress_file('val_cam_result.zip', './val_cam_result')
compress_file('test_cam_result.zip', './test_cam_result')